In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
import torchvision
from model2 import classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test, train_fashion,test_fashion
from federated_learning import distribute_global_model, federated_averaging
from model4 import MultilayerPerceptron
import cluster
from autoencoder2 import Autoencoder2,reduce_dimensions2

In [2]:
# Predefined stuff

n_epochs = 5
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0,), (1.0,))
])

# Load Kuzushiji-MNIST dataset
train_dataset = torchvision.datasets.KMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.KMNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
train_loader_pca = copy.copy(train_loader)
test_loader_pca = copy.copy(test_loader)

train_loader_auto = copy.copy(train_loader)
test_loader_auto = copy.copy(test_loader)

In [5]:
for images, _ in train_loader_auto:
    print("Input image shape:", images.shape)
    break

Input image shape: torch.Size([64, 1, 28, 28])


In [6]:
for images, _ in train_loader_auto:
    print("Min:", images.min().item(), "Max:", images.max().item())  # Should be 0 to 1
    break

Min: 0.0 Max: 1.0


In [7]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1] 

# PCA

In [35]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:
    train_data.append(data.view(data.size(0), -1))  
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  
train_labels = torch.cat(train_labels, dim=0)

train_data_np = train_data.numpy()

pca = PCADigitReducer(100)
train_data_reduced = pca.fit_transform(train_data_np)  

train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

train_data_reconstructed = train_data_reconstructed.view(-1, 1, 28, 28)

train_data_reconstructed = (train_data_reconstructed - 0.2860) / 0.3204

batch_size_train = train_loader_pca.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

# Autoencoder

In [ ]:
# Autoencoder
latent_dim = 100  
autoencoder = Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5
for epoch in range(auto_num_epochs): 
    for images, _ in train_loader_auto:
        auto_optimizer.zero_grad()
        reconstructed = autoencoder(images)
        loss = auto_criterion(reconstructed, images)  
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item()}")

Epoch [1/5], Loss: 0.8595552444458008
Epoch [2/5], Loss: 0.8663073778152466
Epoch [3/5], Loss: 0.8553293347358704
Epoch [4/5], Loss: 0.8085049986839294
Epoch [5/5], Loss: 0.8072602152824402


In [31]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

# Autoencoder 2

In [8]:
autoencoder2 = Autoencoder2()
auto2_criterion = nn.BCELoss()  
auto2_optimizer = optim.Adam(autoencoder2.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
dummy_input = torch.randn(1, 1, 28, 28).to(device)
dummy_output = autoencoder2.encoder(dummy_input)
print("Encoder output shape:", dummy_output.shape)  # Should be (1, 2)


Encoder output shape: torch.Size([1, 100])


In [11]:
dummy_latent = torch.randn(1, 100).to(device)
dummy_reconstruction = autoencoder2.decoder(dummy_latent)
print("Decoder output shape:", dummy_reconstruction.shape)  # Should be (1, 1, 28, 28)


Decoder output shape: torch.Size([1, 1, 28, 28])


In [12]:

num_epochs = 5
for epoch in range(num_epochs):
    autoencoder2.train()
    train_loss = 0

    for images, _ in train_loader_auto:
        images = images.to(device)

        auto2_optimizer.zero_grad()
        outputs = autoencoder2(images)
        loss = auto2_criterion(outputs, images)  # Reconstruction loss
        loss.backward()
        auto2_optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader_auto)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}")

Epoch [1/5], Loss: 0.2230
Epoch [2/5], Loss: 0.1745
Epoch [3/5], Loss: 0.1628
Epoch [4/5], Loss: 0.1562
Epoch [5/5], Loss: 0.1527


In [13]:
autoencoder2.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions2(train_loader_auto, autoencoder2.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder2.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

# Partition

# Classic Dataloaders

In [37]:
# classic
trainingset = train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

In [38]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic_clustered = clustered_data

In [39]:
# Normal loader classic
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [40]:
# Clustered loader classic
classic_client_loaders_clustered = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic_clustered.values()
]

In [41]:
print(len(classic_client_loaders_clustered))

4


# PCA Dataloaders

In [42]:
# pca 
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)

In [43]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca_clustered = clustered_data

In [44]:
# Normal loader pca
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [45]:
# Clustered loader pca
pca_client_loaders_clustered = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca_clustered.values()
]

In [46]:
print(len(pca_client_loaders_clustered))

4


# Autoencoder Dataloader

In [14]:
# Autoencoder
trainingset_auto = reduced_train_loader_auto.dataset
print(trainingset_auto.targets)
print(trainingset_auto.data.shape)
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)

tensor([4, 3, 4,  ..., 1, 4, 2])
torch.Size([60000, 1, 28, 28])


In [15]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_auto.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_auto, targets, num_classes)

partitioned_data_auto_clustered = clustered_data

In [16]:
# Normal loader autoencoder
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [17]:
# Clustered loader autoencoder
auto_client_loaders_clustered = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto_clustered.values()
]

In [18]:
print(len(auto_client_loaders_clustered))

4


# Models

In [19]:
# Weak model
trial_model = classification_model()
trial_model_pca = classification_model()
trial_model_auto = classification_model()

global_model_pca = classification_model()
global_model_auto = classification_model()
global_model_classic = classification_model()

num_clients = 4
# classic models
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

In [20]:
# Stronger model

trial_model_strong = MultilayerPerceptron()
trial_model_pca_strong = MultilayerPerceptron()
trial_model_auto_strong = MultilayerPerceptron()

global_model_pca_strong = MultilayerPerceptron()
global_model_auto_strong = MultilayerPerceptron()
global_model_classic_strong = MultilayerPerceptron()

num_clients = 4
# classic models
local_models_classic_strong = [copy.deepcopy(global_model_classic_strong) for _ in range(num_clients)]
# pca models 
local_models_pca_strong = [copy.deepcopy(global_model_pca_strong) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder_strong = [copy.deepcopy(global_model_auto_strong) for _ in range(num_clients)]

# Training

In [54]:
# test for errors

# Classic weak

optimizer = optim.SGD(trial_model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model, train_loader, optimizer, log_interval, train_losses, train_counter)

c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324756
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.287488
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.306635
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.313749
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.309102
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.301703
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.275743
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.278663
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.296479
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.300802
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.306334
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.279788
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.281782
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.286427
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.297249
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.265355
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.266788
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.282426
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.238970
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.222788
Train Epoch: 1 [12800/60000 (

In [55]:
test_losses_classic_weak = []
test(trial_model,test_loader,test_losses_classic_weak)

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.8018, Accuracy: 7417/10000 (74%)



In [56]:
# Pca weak

optimizer = optim.SGD(trial_model_pca.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_pca, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.328599
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.273793
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.299218
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.249446
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.319825
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.208917
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.255319
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.233950
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.182319
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.195847
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.100594
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.140197
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.091429
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.048855
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.080810
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.964301
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.901302
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.857477
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.820555
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.859565
Train Epoch: 1 [12800/60000 (

In [57]:
test_losses_classic_pca = []
test(trial_model_pca,train_loader_reduced_pca,test_losses_classic_pca)


Test set: Avg. loss: 0.2647, Accuracy: 55526/60000 (93%)



In [21]:
# Auto weak

optimizer = optim.SGD(trial_model_auto.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_auto, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306548
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.317788
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.298679
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.310241
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.310212
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.301140
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.304917
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.321371
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.302783
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.308134
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.299691
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.292048
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.297222
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.292580
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.294728
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.298414
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.297703
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.294195
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.269562
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.283832
Train Epoch: 1 [200

In [22]:
test_losses_classic_auto = []
test(trial_model_auto,reduced_train_loader_auto,test_losses_classic_auto)

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.4895, Accuracy: 51589/60000 (86%)



In [60]:
# Classic strong

optimizer = optim.SGD(trial_model_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_strong, train_loader, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304227
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.303013
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.288105
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.291003
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.294431
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.282763
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.282045
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.252639
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.276391
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.245316
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.238302
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.244856
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.234973
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.216369
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.193696
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.210846
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.178809
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.173445
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.158132
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.160153
Train Epoch: 1 [12800/60000 (

In [61]:
test_losses_classic_strong = []
test_fashion(trial_model_strong,test_loader,test_losses_classic_strong)


Test set: Avg. loss: 0.7021, Accuracy: 7846/10000 (78%)



In [62]:
# Pca strong

optimizer = optim.SGD(trial_model_pca_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_pca_strong, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307136
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290158
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.236774
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.183823
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.152716
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.129555
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.976023
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.927251
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.779624
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.729397
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.532936
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.352069
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.324806
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.168715
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.210694
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.094947
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.031656
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.011030
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.032438
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.799944
Train Epoch: 1 [12800/60000 (

In [63]:
test_losses_pca_strong = []
test_fashion(trial_model_pca_strong,train_loader_reduced_pca,test_losses_pca_strong)


Test set: Avg. loss: 0.1734, Accuracy: 57029/60000 (95%)



In [23]:
# Auto strong

optimizer = optim.SGD(trial_model_auto_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_auto_strong, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308955
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.310260
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.311513
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.292296
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.283921
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.291498
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.284542
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.293563
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.263078
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.251604
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.251185
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.256899
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.230090
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.239281
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.204912
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.173196
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.193265
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.165194
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.142742
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.150473
Train Epoch: 1 [200

In [24]:
test_losses_auto_strong = []
test_fashion(trial_model_auto_strong,reduced_train_loader_auto,test_losses_auto_strong)


Test set: Avg. loss: 0.4526, Accuracy: 51749/60000 (86%)



In [26]:
# Autoencoder

rounds_auto = 20

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder_strong,single=False)

    distribute_global_model(global_weights_auto, global_model_auto_strong,single=True)
    test_losses = []
    test_fashion(global_model_auto_strong,test_loader_auto,test_losses)

Round 1/20
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.332955
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.204590
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.245976
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.186005
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.205507
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.209065
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.186772
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.224621
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.332909
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.210489
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.279403
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.372140
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.282287
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.346442
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.202239
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.154833
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.176950
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.268171
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.242183
Train Epoch: 1 [19000/23974 (79%)]	Lo